In [ ]:
import plotly
import plotly.express as px
import pandas as pd
from config import postgresPass as pw
from sqlalchemy import create_engine

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import func, inspect

In [ ]:
protocol = 'postgresql'
username = 'postgres'
password = pw
host = 'localhost'
port = 5432
database_name = 'bike_shop_4'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine,reflect=True)

In [ ]:
# View all of the classes that automap found
Base.classes.keys()

In [ ]:
inspector = inspect(engine)
inspector.get_table_names()

In [ ]:
# save list of column name lists and types for tables
customers_columns = inspector.get_columns('customers')
combined_sales_columns = inspector.get_columns('combined_sales')
calendar_columns = inspector.get_columns('calendar')
products_columns = inspector.get_columns('products')
categories_columns = inspector.get_columns('categories')
returns_columns = inspector.get_columns('returns')
territories_columns = inspector.get_columns('territories')

In [ ]:
for c in calendar_columns:
    print(c['name'], c["type"])

In [ ]:
# Save references to each table
customers = Base.classes.customers
sales = Base.classes.combined_sales
calendar = Base.classes.calendar
products = Base.classes.products
categories = Base.classes.categories
returns = Base.classes.returns
territories = Base.classes.territories

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [ ]:
# Design a query to calculate the total number customers in each Education Level
result1 = session.query(customers.educationlevel, func.count(customers.educationlevel)).\
    group_by(customers.educationlevel).order_by(func.count(customers.educationlevel).desc()).all()

In [ ]:
df1 = pd.DataFrame(result1)

In [ ]:
df1.rename(columns={0: 'Education Level'},inplace=True)

In [ ]:
df1.rename(columns={1: 'Quantity'},inplace=True)

In [ ]:
px.bar(df1, x='Education Level', y='Quantity')

In [ ]:
# Design a query to calculate the total number customers in each Annual Income bracket
result2 = session.query(customers.annualincome, func.count(customers.annualincome)).\
    group_by(customers.annualincome).order_by(func.count(customers.annualincome).desc()).all()

In [ ]:
df2 = pd.DataFrame(result2)

In [ ]:
df2.rename(columns={1: 'Quantity'},inplace=True)

In [ ]:
df2.rename(columns={0: 'Income Level'},inplace=True)

In [ ]:
px.bar(df2, x='Income Level', y='Quantity')

In [ ]:
# Design a query to calculate the total number of products in each product sub category
result3 = session.query(products.productsubcategorykey, func.count(products.productsubcategorykey)).\
    group_by(products.productsubcategorykey).order_by(products.productsubcategorykey).all()

In [ ]:
df3 = pd.DataFrame(result3)

In [ ]:
df3.rename(columns={1: 'Quantity'},inplace=True)

In [ ]:
df3.rename(columns={0: 'Product Subcategory'},inplace=True)

In [ ]:
px.bar(df3, x='Product Subcategory', y='Quantity')

In [ ]:
products_join = pd.read_sql_query(
    """
    SELECT
    p.productkey,
    p.productsubcategorykey,
    p.modelname,
    cs.productkey,
    cs.orderquantity,
    cs.customerkey
    FROM products AS p
    INNER JOIN combined_sales AS cs
    ON p.productkey = cs.productkey;
  
     
    """,
    con=engine)

In [ ]:
# Get all the data for Bikes. 
just_bikes_df01 = products_join.loc[products_join["productsubcategorykey"] <= 3]

In [ ]:
just_bikes_df01.pop('productkey')

In [ ]:
just_bikes_df01

In [ ]:
# Get model name data
model_data = just_bikes_df01["modelname"].value_counts()
md1 = pd.DataFrame(model_data)
md1.reset_index()

In [ ]:
md1.rename(columns={'modelname': 'Quantity'},inplace=True)

In [ ]:
px.bar(md1, x=md1.index, y='Quantity')

In [ ]:
customers_df = pd.read_sql_query(
    """
    SELECT
    customerkey,
    gender,
    annualincome,
    educationlevel
    FROM customers
     
    """,
    con=engine)

In [ ]:
df4 = pd.merge(just_bikes_df01,customers_df, how="left",on='customerkey')

In [ ]:
df4

In [ ]:
# Get Education Level data
Education_data = df4["educationlevel"].value_counts()
ed1 = pd.DataFrame(Education_data)
ed1.reset_index()
ed1.rename(columns={'educationlevel': 'Quantity'},inplace=True)
px.bar(ed1, x=ed1.index, y='Quantity')

In [ ]:
# Get Annual Income Level data
annualincome_data = df4["annualincome"].value_counts()
ai1 = pd.DataFrame(annualincome_data)
ai1.reset_index()
ai1.rename(columns={'annualincome': 'Quantity'},inplace=True)
px.bar(ai1, x=ai1.index, y='Quantity')

In [ ]:
first_table = pd.read_sql_query(
    """
    SELECT
    t.country,
    cs.orderquantity
    FROM territories t
    INNER JOIN combined_sales AS cs
    ON t.salesterritorykey = cs.salesterritorykey;
     
    """,
    con=engine)


In [ ]:
first_table1 = first_table.groupby(by="country", dropna = False).sum()

In [ ]:
first_table1

In [ ]:
# Get Country Level data
cd1 = pd.DataFrame(first_table1)
cd1.reset_index()
cd1.rename(columns={'orderquantity': 'Quantity'},inplace=True)
px.bar(cd1, x=cd1.index, y='Quantity')

In [ ]:
second_table = pd.read_sql_query(
    """
    SELECT
    p.productname,
    r.returnquantity
    FROM products p
    INNER JOIN returns r
    ON p.productkey = r.productkey;
     
    """,
    con=engine)
second_table.head(10)

In [ ]:
px.bar(second_table, x='productname', y='returnquantity')

In [ ]:
third_table = pd.read_sql_query(
    """
    SELECT
    c.subcategoryname,
    p.productcost,
    p.productprice
    FROM categories c
    INNER JOIN products p
    ON c.productsubcategorykey = p.productsubcategorykey;
     
    """,
    con=engine)
third_table.head()

In [ ]:
px.bar(third_table, x='subcategoryname', y='productprice')

In [ ]:
forth_table = pd.read_sql_query(
    """
    SELECT
    c.totalchildren,
    cs.orderquantity
    FROM customers c
    INNER JOIN combined_sales cs
    ON c.customerkey = cs.customerkey;
     
    """,
    con=engine)
forth_table.head()

In [ ]:
px.pie(forth_table, 'totalchildren', 'orderquantity')